In [1]:
import numpy as np
import pandas as pd
import csv
from scipy.special import logsumexp
from scipy.stats import multivariate_normal as mvn

In [7]:
data =[]
line_count = 0
with open('Rainier_Weather.csv',newline='') as file:
    csvFile = csv.reader(file,delimiter=',')
    for lines in csvFile:
        if line_count>0:
            data.append([float(a) for a in lines[1:]])
        line_count+=1
data = np.array(data)

In [13]:
n_states = 3
n_obs = data.shape[0]
dim_data = data.shape[1]


transition = np.random.normal(0, 1, size=(n_states, n_states))
log_transition = transition - logsumexp(transition, axis=1, keepdims=True)

log_emission = np.log(np.random.random((n_obs, n_states)))

log_forward = np.random.rand(n_obs,n_states)

log_backward = np.random.rand(n_obs,n_states)

log_init_prob = np.log(np.ones(n_states)/n_states)


p_old = -10000
tol = 0.001
max_iter = 100

means  = np.random.normal(0, 1, size=(n_states, dim_data))
covars = np.empty((n_states, dim_data, dim_data))

In [17]:
def baum_welch(obs, states, n_iters):
    """EM for hidden Markov models, i.e. the Baum–Welch algorithm. Numerical 
    instability handled by working in log space.
    """
    N, D = obs.shape
    K = len(states)

    # Initialize parameters \theta:
    #
    # 1. Probaiblities \pi (size K).
    # 2. Transition probability matrix A.
    # 3. Emission parameters \phi (Gaussian case: \mu and \Sigma).
    #
    log_pi = np.ones(K) / K
    assert np.isclose(log_pi.sum(), 1)

    log_A  = np.random.normal(0, 1, size=(K, K))
    log_A -= logsumexp(log_A, axis=1, keepdims=True)
    assert np.allclose(np.sum(np.exp(log_A), axis=1), 1)

    means  = np.random.normal(0, 1, size=(K, D))
    covars = np.empty((K, D, D))
    for k in range(K):
        # Ensure initial covariance matrices are PSD.
        tmp = np.random.random((D, 2*D))
        covars[k] = tmp @ tmp.T

    # Initialize emission probabilities (size N × K).
    log_emm_prob = np.random.random((N, K))

    # The n-th row is log(alpha(z_n)).
    # The k-th column is value z_n takes.
    # So (nk)-th cell is alpha(z_n = k).
    log_alpha = np.zeros((N, K))
    log_beta  = np.zeros((N, K))

    for _ in range(n_iters):

        # E-step (forward-backward algorithm).
        # ------------------------------------
        for k in range(K):
            log_alpha[0, k] = log_pi[k] + log_emm_prob[0, k]

        for n in range(1, N):
            for k in range(K):
                tmp = np.empty(K)
                for j in range(K):
                    tmp[j] = log_alpha[n-1, j] + log_A[j, k]
                log_alpha[n, k] = logsumexp(tmp) + log_emm_prob[n, k]

        log_beta[N-1] = 0  # log(1)
        for n in reversed(range(N-1)):
            for k in range(K):
                tmp = np.empty(K)
                for j in range(K):
                    tmp[j] = (log_beta[n+1, j] 
                              + log_emm_prob[n+1, j] 
                              + log_A[k, j])
                log_beta[n, k] = logsumexp(tmp)

        # M-step.
        # ------------------------------------

        # Compute first posterior moment, \gamma (size N × K).
        # Eq. 13.33 in Bishop.
        log_gamma = log_alpha + log_beta
        log_evidence = logsumexp(log_alpha[N-1])
        gamma = np.exp(log_gamma - log_evidence)
        assert np.allclose(np.sum(gamma, axis=1), 1)

        # Compute second posterior moment, \xi (size N × K × K). For the n-th 
        # sample, the (K × K) matrix A is defined such that 
        # A_{ij} = E[z_{n} = i, z_{n+1} = j].
        #
        # Eq. 13.43 in Bishop.
        log_xi = np.empty((N, K, K))
        for n in range(N-1):
            tmp = np.empty((K, K))
            for k in range(K):
                for j in range(K):
                    tmp[k, j] = (log_alpha[n, k]
                                 + log_beta[n+1, j] 
                                 + log_emm_prob[n+1, j] 
                                 + log_A[k, j]
                                 - log_evidence)
            log_xi[n] = tmp

        # Eq. 13.18 in Bishop.
        log_pi = log_gamma[0] - logsumexp(log_gamma[0])
        assert log_pi.size == K
        assert np.isclose(np.sum(np.exp(log_pi)), 1)

        # Eq. 13.19 in Bishop.
        for i in range(K):
            for j in range(K):
                log_A[i, j] = logsumexp(log_xi[1:, i, j]) - logsumexp(log_xi[1:, i, :])
        assert np.allclose(np.sum(np.exp(log_A), axis=1), 1)

        # Use matrix multiplication to sum over N.
        # Eq. 13.20 in Bishop.
        for k in range(K):
            means[k] = obs.T @ gamma[:, k]
            means[k] /= gamma[:, k].sum()

        # Compute new covariances.
        # Eq. 13.21 in Bishop.
        for k in range(K):
            covars[k] = np.zeros((D, D))
            for n in range(N):
                dev = obs[n] - means[k]
                covars[k] += gamma[n, k] * np.outer(dev, dev.T)
            covars[k] /= gamma[:, k].sum()

        # Recompute emission probabilities using inferred states.
        for n in range(N):
            x = obs[n]
            for k in range(K):
                mu = means[k]
                # var = covars[k] + 100 * np.eye(D)
                M = np.random.random((N, D))
                var = M.T @ M
                log_emm_prob[n, k] = mvn.logpdf(x, mu, var)

    Z = np.argmax(gamma, axis=1)
    theta = (np.exp(log_pi), np.exp(log_A), means, covars)
    return Z, theta

In [20]:
Z,theta = baum_welch(data, [1,1,1], 100)

In [ ]:
for ite in range(max_iter):


    log_forward[0,i] = log_init_prob + log_emission[0,:]
    
    for t in range(n_obs-1):
        log_forward[t+1,:] = np.matmul(log_forward[t,:] ,transition) + log_emission[t+1,:]

    backward[-1,:] = 1
    for t in reversed(range(size-1)):
        temp = np.matmul(backward_hat[t+1,:]*emission[:,data[t+1]],np.transpose(transition))
        backward[t,:] = 


    a = np.zeros((size,n_states))
    b = np.zeros((size,n_states,n_states))
    for i in range(size):
        for j in range(n_states):
            a[i,j] = forward_hat[i,j]*backward_hat[i,j]
    for t in range(size-1):
        for i in range(n_states):
            for j in range(n_states):
                b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j]*transition[i,j]*emission[j,data[t+1]]

    for i in range(n_states):
        for j in range(n_states):
            transition[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])

    for i in range(n_states):
        init_prob[i] = a[0,i]/np.sum(a[0,:])

    for i in range(n_states):
        for j in range(n_obs):
            emission[i,j] = np.sum(a[np.argwhere(data==j),i]) / np.sum(a[:,i])
            
    p = np.sum(np.log(scale_factors))
    print(f'p is:{p}')
    print(f'transition prob is: {transition}')
    print(f'emission prob is:{emission}')
    # if p - p_old < tol:
    #     break
    p_old = p

In [2]:
x = np.array([1000, 1000, 1000])
# def logsumexp(x):
#     # c = x.max()
#     c = 0
#     return c + np.log(np.sum(np.exp(x - c)))

In [ ]:
np.exp(x - logsumexp(x))